# Taking a brief look at the fire dataset

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMapWithTime
from datetime import datetime, timedelta
import os
import numpy as np

## Read the archival MODIS data

In [ ]:
df = pd.read_csv("dataset/raw_data/fire_archive_M6_96619.csv")
df.sort_values("acq_date").head()

In [ ]:
df.sort_values("acq_date").tail()

In [ ]:
df.shape

## Drop unused columns
We only need lat, long, date, and confidence for our visualization

In [ ]:
df.drop(['scan', 'track', 'acq_time', 'satellite', 'instrument', 'version', 'brightness', 'bright_t31', 'frp', 'type'], axis=1, inplace=True)

# get min and max confidence values
max_conf = df['confidence'].max()
min_conf = df['confidence'].min()
print(max_conf)
print(min_conf)

In [ ]:
df.head()

## Split the data into days & scale weights for heatmap

In [ ]:
days = []
for month in range(8, 10):
    for day in range(1, 32):
        # create a date string in the same format that the dataset uses
        daystr = f'2019-0{month}-{day}'
        if day < 10:
            # path day with zeros if necessary
            daystr = f'2019-0{month}-0{day}'

        d = df[df["acq_date"] == daystr][["latitude", "longitude", "confidence"]]
        days.append(d)

# the last day in the list is empty because sepetember is only 30 days long
days.pop()

In [ ]:
# This should be september 30th, and should not be empty
print(days[-1])

## Prepare the data for the heatmap
We need to scale the confidence values to [0, 1] because the weights for a folium heatmap must be within this range

In [ ]:
def scale(x, min, max):
    return (x - min) / (max - min)

data = []
for d in days:
    scaled = d.copy()
    scaled['confidence'] = scaled['confidence'].apply(scale, args=(min_conf, max_conf))
    data.append(scaled.values.tolist())

data[0]

## Create a time index list so that folium knows what date to put on each step

In [ ]:
start_date = datetime(year=2019, month=8, day=1)
# generate the proper date string for each day
time_index = [(start_date + k * timedelta(1)).strftime("%Y-%m-%d") for k in range(len(data))]

## Make a heatmap of confidence values over coordinates

In [ ]:
m = folium.Map(
    [-24.0, 140.0],
    zoom_start=4,
)

hm = HeatMapWithTime(data, index=time_index, auto_play=True)
hm.add_to(m)

os.makedirs('graphs/', exist_ok=True)
m.save('graphs/fire_archive_M6_96619.html')